In [1]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 5.4 MB/s eta 0:00:00


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth


client_id = '5b0dffb8f4d04048a43b57b22472b988'
client_secret = '02ec75f99deb496f9c76bc9608406e12'
redirect_uri = 'http://localhost:8888/callback'

sp = spotipy.Spotify(auth_manager= SpotifyOAuth(
    client_id= client_id,
    client_secret= client_secret,
    redirect_uri= redirect_uri,
    scope= 'user-top-read',
    open_browser= False
))

In [3]:
listening_history = sp.current_user_top_tracks(limit=50, offset=0, time_range='medium_term')
artist_history = sp.current_user_top_artists(limit=50, offset=0, time_range='medium_term')

Go to the following URL: https://accounts.spotify.com/authorize?client_id=5b0dffb8f4d04048a43b57b22472b988&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=user-top-read
Enter the URL you were redirected to: http://localhost:8888/callback?code=AQCq-ecUQo4Eq8hgsQy3BYB54pgAxOYc6a9Xw-SCAn6EcCeiIVys9H49WwtCKv4k5n1blzis4GDNYm93hKIZX7US3dquZZrN7MnFRBAIutVmEoBWnWjE6KWgRZVVsr5zmWsW5jDFk72cFaIJhG6x_krBejk-PnANLPtxWfUhLZZCRkRSbypkhSpVZkaL4038mg


In [4]:
import pandas as pd

In [10]:
for key in listening_history.keys():
    print(key)

print('----')
for feature in listening_history['items'][0].keys():
    print(feature)


items
total
limit
offset
href
next
previous
----
album
artists
available_markets
disc_number
duration_ms
explicit
external_ids
external_urls
href
id
is_local
is_playable
name
popularity
preview_url
track_number
type
uri


In [9]:
for key in artist_history.keys():
    print(key)

print('----')
for feature in artist_history['items'][0].keys():
    print(feature)

items
total
limit
offset
href
next
previous
----
external_urls
followers
genres
href
id
images
name
popularity
type
uri


In [11]:
pd.json_normalize(artist_history)

,items,total,limit,offset,href,next,previous
0,[{'external_urls': {'spotify': 'https://open.s...,204,50,0,https://api.spotify.com/v1/me/top/artists?offs...,https://api.spotify.com/v1/me/top/artists?offs...,None


In [12]:
df_artists = pd.json_normalize(artist_history['items'])
artist_features = ['genres', 'id', 'name', 'popularity', 'followers.total']
df_artists = df_artists[artist_features]
df_artists = df_artists.explode(column= 'genres').reset_index(drop=True)
df_artists

,genres,id,name,popularity,followers.total
0,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,71264
1,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
2,witch house,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
3,lo-fi house,6G01WYFYF91rjG5LtwMhY4,1tbsp,46,32378
4,midwest emo,2CnhqfjUG0qzsru0SMuhrk,Title Fight,63,514218
...,...,...,...,...,...
112,hard house,5ye3YheRaIaodo3nid7Wxz,Upper90,40,7930
113,eurodance,5ye3YheRaIaodo3nid7Wxz,Upper90,40,7930
114,trance,5ye3YheRaIaodo3nid7Wxz,Upper90,40,7930
115,cloud rap,5tQMB0cuNXdCtzovGt55uD,LUCKI,77,1450295


In [13]:
while artist_history['next'] != None:
  artist_history = sp.next(artist_history)
  temp_artists = pd.json_normalize(artist_history['items'])
  temp_artists = temp_artists.explode(column= 'genres').reset_index(drop=True)
  temp_artists = temp_artists[artist_features]
  df_artists = pd.concat([df_artists, temp_artists], ignore_index=True)

df_artists

,genres,id,name,popularity,followers.total
0,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,71264
1,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
2,witch house,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
3,lo-fi house,6G01WYFYF91rjG5LtwMhY4,1tbsp,46,32378
4,midwest emo,2CnhqfjUG0qzsru0SMuhrk,Title Fight,63,514218
...,...,...,...,...,...
394,hip hop,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,76,11524392
395,east coast hip hop,7BMccF0hQFBpP6417k1OtQ,Action Bronson,61,851326
396,jazz rap,04aAAEbkuy8tC8xmaE0CcQ,Nitsua,41,16177
397,NaN,1ybINI1qPiFbwDXamRtwxD,Smino,66,1005430


In [14]:
df_artists.rename(columns= {'genres': 'subgenre'}, inplace= True)
df_artists

,subgenre,id,name,popularity,followers.total
0,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,71264
1,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
2,witch house,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
3,lo-fi house,6G01WYFYF91rjG5LtwMhY4,1tbsp,46,32378
4,midwest emo,2CnhqfjUG0qzsru0SMuhrk,Title Fight,63,514218
...,...,...,...,...,...
394,hip hop,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,76,11524392
395,east coast hip hop,7BMccF0hQFBpP6417k1OtQ,Action Bronson,61,851326
396,jazz rap,04aAAEbkuy8tC8xmaE0CcQ,Nitsua,41,16177
397,NaN,1ybINI1qPiFbwDXamRtwxD,Smino,66,1005430


In [16]:
df_artists['subgenre'].unique()

array(['cloud rap', 'electroclash', 'witch house', 'lo-fi house',
       'midwest emo', 'shoegaze', 'post-hardcore', 'melodic hardcore',
       'pop punk', 'darkwave', 'post-punk', 'cold wave', 'eurotrance',
       'hard house', 'eurodance', 'techno', 'hyperpop', 'phonk',
       'hard techno', 'acid techno', 'tekno', 'jazz rap', 'stutter house',
       'house', 'dark trap', 'memphis rap', 'underground hip hop',
       'emo rap', 'trance', nan, 'nu jazz', 'reggae', 'roots reggae',
       'melodic rap', 'dubstep', 'edm', 'electro', 'electronic', 'punk',
       'rock', 'emo', 'skate punk', 'breakcore', 'jungle', 'speedcore',
       'breakbeat', 'drum and bass', 'reggae rock', 'ska punk', 'ska',
       'rage rap', 'bedroom pop', 'indie pop', 'hip hop', 'rap',
       'punk rap', 'melodic house', 'progressive house', 'deep house',
       'french house', 'riddim', 'bass music', 'deathstep', 'rave', 'dub',
       'nu metal', 'alternative metal', 'rap metal', 'psytrance',
       'experimental h

In [19]:
rap_names = ['Isaiah Rashad', 'Kevin Gates', 'Freddie Gibbs', 'Beast Coast',
             'Mac Miller', 'The Underachievers', 'ScHoolboy Q', 'Tay-K',
             'Vince Staples','Rae Sremmurd', 'Dave East', 'Logic', 'Cordae',
             'Riff Raff', 'Flatbush Zombies', 'Kodak Black', 'Smino']

indie_names = ['Vacations', 'Eyedress', 'WILLIS', 'The Simps',
               'Chance the Rapper','TV Girl', 'Palace', 'Golden Vessel',
               'Tyler, The Creator', 'Oliver Tree', 'Mind’s Eye',
               'Gracie Abrams', 'Steve Lacy', 'Current Joys']

electronic_names = ['KAYTRANADA', 'Macabre Plaza', 'LoVibe.']
house_names = ['Mietze Conte', '49th & Main', 'BL3SS', 'Sango']
ambient_names = ['C418']
jazz_names = ['OMA']
soul_names = ['Free Nationals', 'Bruno Mars', 'Thundercat']

na_mapping = {
    'rap': rap_names,
    'indie': indie_names,
    'electronic': electronic_names,
    'house': house_names,
    'ambient': ambient_names,
    'nu jazz': jazz_names,
    'soul': soul_names

}


for genre, artists in na_mapping.items():
  df_artists.loc[df_artists['name'].isin(artists), 'subgenre'] = df_artists['subgenre'].fillna(genre)

df_artists.tail(10)

,subgenre,id,name,popularity,followers.total
389,hyperpop,3xMSStWH6Wz2KAJqkZwXLu,Yabujin,46,160470
390,electro house,4Ws2otunReOa6BbwxxpCt6,Benny Benassi,64,580566
391,psychedelic pop,4kSGbjWGxTchKpIxXPJv0B,Crumb,61,616881
392,gangster rap,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,76,11524392
393,east coast hip hop,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,76,11524392
394,hip hop,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,76,11524392
395,east coast hip hop,7BMccF0hQFBpP6417k1OtQ,Action Bronson,61,851326
396,jazz rap,04aAAEbkuy8tC8xmaE0CcQ,Nitsua,41,16177
397,rap,1ybINI1qPiFbwDXamRtwxD,Smino,66,1005430
398,soundtrack,3nIELxtRA56U1AXqJFzyDw,Yuka Kitamura,52,42024


In [20]:
df_artists.rename(columns= {'id': 'artist_id', 'name': 'artist', 'followers.total': 'followers'}, inplace= True)
df_artists

,subgenre,artist_id,artist,popularity,followers
0,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,71264
1,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
2,witch house,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
3,lo-fi house,6G01WYFYF91rjG5LtwMhY4,1tbsp,46,32378
4,midwest emo,2CnhqfjUG0qzsru0SMuhrk,Title Fight,63,514218
...,...,...,...,...,...
394,hip hop,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,76,11524392
395,east coast hip hop,7BMccF0hQFBpP6417k1OtQ,Action Bronson,61,851326
396,jazz rap,04aAAEbkuy8tC8xmaE0CcQ,Nitsua,41,16177
397,rap,1ybINI1qPiFbwDXamRtwxD,Smino,66,1005430


In [21]:
genre_mapping = {
    'Hip Hop':['east coast hip hop', 'boom bap','g-funk', 'hardcore hip hop',
               'west coast hip hop', 'gangster rap', 'crunk', 'southern hip hop'],

    'Rap': ['indie rap','experimental hip hop','trap', 'drill', 'horrorcore',
            'underground hip hop', 'memphis rap', 'cloud rap', 'emo rap',
            'melodic rap', 'rap metal', 'rage rap', 'hip hop', 'rap', 'dark trap'],

    'House': ['tech house','future house', 'afro house','french house',
              'stutter house', 'house', 'deep house', 'hard house',
              'melodic house', 'lo-fi house', 'progressive house',
              'electro house', 'piano house'],

    'Dance': ['psytrance','uk funky', 'trance', 'phonk', 'eurodance', 'edm',
              'eurotrance', 'italo dance', 'happy hardcore', 'brazilian funk',
              'kuduro', 'alternative dance'],

    'Electronic': ['indietronica', 'electroclash', 'witch house', 'riddim',
                   'bass music', 'new rave', 'rave', 'dub', 'dubstep',
                   'speedcore', 'drum and bass', 'breakcore', 'hyperpop',
                   'jungle', 'electronic', 'deathstep', 'breakbeat', 'electro',
                   'uk garage', 'electronica'],

    'Ambient': ['ambient', 'idm'],

    'Techno': ['hard techno', 'acid techno', 'tekno', 'techno', 'melodic techno'],

    'Alternative Rock': ['metalcore', 'post-grunge', 'slowcore', 'shoegaze',
                         'midwest emo', 'darkwave', 'coldwave', 'emo',
                         'alternative metal', 'post-hardcore',
                         'melodic hardcore', 'nu metal', 'cold wave'],

    'Jazz': ['nu jazz', 'jazz funk', 'jazz fusion', 'jazz rap'],

    'Latin': ['latin', 'urbano latino', 'trap latino', 'merengue',
              'música tropical', 'reggaeton'],

    'Soul': ['retro soul', 'soul'],

    'Video Game': ['soundtrack'],

    'Punk': ['post-punk', 'skate punk', 'punk', 'pop punk', 'punk rap'],

    'Rock': ['rock', 'reggae rock', 'electronic rock', 'hard rock',
             'acid rock', 'psychedelic rock', 'blues rock', 'soft rock',
             'classic rock'],

    'Reggae': ['ska', 'ska punk', 'reggae', 'roots reggae'],

    'Indie': [None, 'indie pop', 'bedroom pop', 'psychedelic pop', 'indie',
              'britpop', 'dream pop']
     }

In [23]:
def map_genre(subgenres):
    return next((genre for genre, subgenre_list in genre_mapping.items() if any(subgenre in subgenre_list for subgenre in subgenres)), None)

df_artists['genre'] = df_artists['subgenre'].apply(lambda x: map_genre([x]) if isinstance(x, str) else None)
df_artists.head()

,subgenre,artist_id,artist,popularity,followers,genre
0,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,71264,Rap
1,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706,Electronic
2,witch house,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706,Electronic
3,lo-fi house,6G01WYFYF91rjG5LtwMhY4,1tbsp,46,32378,House
4,midwest emo,2CnhqfjUG0qzsru0SMuhrk,Title Fight,63,514218,Alternative Rock


In [24]:
for artist, group in df_artists.groupby('artist'):
  mode_genre = group['genre'].mode()[0]
  df_artists.loc[group.index, 'genre'] = mode_genre

df_artists.drop_duplicates(subset= ['artist_id', 'genre'], inplace= True)
df_artists.reset_index(drop= True, inplace= True)
df_artists

,subgenre,artist_id,artist,popularity,followers,genre
0,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,71264,Rap
1,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706,Electronic
2,lo-fi house,6G01WYFYF91rjG5LtwMhY4,1tbsp,46,32378,House
3,midwest emo,2CnhqfjUG0qzsru0SMuhrk,Title Fight,63,514218,Alternative Rock
4,darkwave,62s5q1tm3UAcRnbGY9CCI9,Provoker,47,63429,Alternative Rock
...,...,...,...,...,...,...
199,gangster rap,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,76,11524392,Hip Hop
200,east coast hip hop,7BMccF0hQFBpP6417k1OtQ,Action Bronson,61,851326,Hip Hop
201,jazz rap,04aAAEbkuy8tC8xmaE0CcQ,Nitsua,41,16177,Jazz
202,rap,1ybINI1qPiFbwDXamRtwxD,Smino,66,1005430,Rap


In [25]:
df_tracks = pd.json_normalize(listening_history['items'])
features = ['artists', 'duration_ms', 'explicit', 'id', 'name', 'popularity', 'album.name', 'album.release_date', 'album.total_tracks']
df_tracks = df_tracks[features]
df_tracks = df_tracks.explode(column= 'artists').reset_index(drop=True)
df_tracks['artist'] = ''
df_tracks['artist_id'] = ''
for idx,row in df_tracks['artists'].items():
  df_tracks.loc[idx, 'artist']= row['name']
  df_tracks.loc[idx, 'artist_id'] = row['id']

df_tracks

,artists,duration_ms,explicit,id,name,popularity,album.name,album.release_date,album.total_tracks,artist,artist_id
0,{'external_urls': {'spotify': 'https://open.sp...,63817,True,3iBtk9V7k32rkvE9bSKJWc,gonmakit,45,gonmakit,2024-05-10,1,Brennan Jones,4PaHlM2H2hC3VQV4BfLo4B
1,{'external_urls': {'spotify': 'https://open.sp...,218500,False,3iRoXGFrF8O0jTul6LAmus,Under Your Spell,78,April Mixtape 3,2023-05-05,16,Snow Strippers,6TsAG8Ve1icEC8ydeHm3C8
2,{'external_urls': {'spotify': 'https://open.sp...,201034,False,3C4qukRl7kX1wdevwbOudf,Vehicle Dissolve,33,Body Jumper,2021-08-13,13,Provoker,62s5q1tm3UAcRnbGY9CCI9
3,{'external_urls': {'spotify': 'https://open.sp...,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,1tbsp,6G01WYFYF91rjG5LtwMhY4
4,{'external_urls': {'spotify': 'https://open.sp...,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,Mietze Conte,3ETvRzzQK7ppB28GcAE8rN
...,...,...,...,...,...,...,...,...,...,...,...
67,{'external_urls': {'spotify': 'https://open.sp...,181545,False,5QOBT97OmYCZo1W5u7tRrB,ten,65,ten,2023-10-13,1,Jozzy,6Ja6zFB5d7XRihhfMo6KzY
68,{'external_urls': {'spotify': 'https://open.sp...,181545,False,5QOBT97OmYCZo1W5u7tRrB,ten,65,ten,2023-10-13,1,Jim Legxacy,7IrBqZo6diq3hV3GpUhrs2
69,{'external_urls': {'spotify': 'https://open.sp...,196347,True,6uDGQOJAI3Ys2DDDIl6Px9,Fight the Vegans,48,Fight the Vegans,2018-06-20,1,WILLIS,3OykSjAE6F2Mb51WplFoRk
70,{'external_urls': {'spotify': 'https://open.sp...,208457,False,0Tcy9RRWvcnqepvmVcqv05,Black Out Days - Subtronics Remix,58,Black Out Days (Subtronics Remix),2023-06-16,1,Phantogram,1l9d7B8W0IHy3LqWsxP2SH


In [26]:
while listening_history['next'] != None:
  listening_history = sp.next(listening_history)
  temp_tracks = pd.json_normalize(listening_history['items'])
  temp_tracks = temp_tracks[features]
  temp_tracks = temp_tracks.explode(column= 'artists').reset_index(drop=True)
  temp_tracks['artist'] = ''
  temp_tracks['artist_id'] = ''
  for idx,row in temp_tracks['artists'].items():
    temp_tracks.loc[idx, 'artist'] = row['name']
    temp_tracks.loc[idx, 'artist_id'] = row['id']
  temp_tracks.drop(columns= 'artists', inplace=True)
  df_tracks = pd.concat([df_tracks, temp_tracks], ignore_index=True)


df_tracks.drop(columns= 'artists', inplace=True)
df_tracks

,duration_ms,explicit,id,name,popularity,album.name,album.release_date,album.total_tracks,artist,artist_id
0,63817,True,3iBtk9V7k32rkvE9bSKJWc,gonmakit,45,gonmakit,2024-05-10,1,Brennan Jones,4PaHlM2H2hC3VQV4BfLo4B
1,218500,False,3iRoXGFrF8O0jTul6LAmus,Under Your Spell,78,April Mixtape 3,2023-05-05,16,Snow Strippers,6TsAG8Ve1icEC8ydeHm3C8
2,201034,False,3C4qukRl7kX1wdevwbOudf,Vehicle Dissolve,33,Body Jumper,2021-08-13,13,Provoker,62s5q1tm3UAcRnbGY9CCI9
3,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,1tbsp,6G01WYFYF91rjG5LtwMhY4
4,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,Mietze Conte,3ETvRzzQK7ppB28GcAE8rN
...,...,...,...,...,...,...,...,...,...,...
3893,341967,False,7k0mvImLkHXeiaAYDQDnyB,"In Europe, Kinda Sad",15,kanashī,2021-03-17,4,1tbsp,6G01WYFYF91rjG5LtwMhY4
3894,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,39,Im_my_worst_enemy,2021-02-01,9,Harto Falión,0wAEYYaHOqYJhHaSvpZUwP
3895,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,39,Im_my_worst_enemy,2021-02-01,9,evilgiane,4bbAeMRdMbfCLVSlpryAVV
3896,112243,True,7tHXd3CsT7VGZAJ5Jkqt61,nothing like uuu,29,nothing like uuu,2024-04-29,1,Nettspend,2jl4qd6UbzeCmImT4nWbtA


In [28]:
df_tracks.rename(columns= {'id': 'track_id', 'name': 'track', 'album.name': 'album', 'album.release_date': 'album_release_date', 'album.total_tracks': 'album_total_tracks'}, inplace=True)
df_tracks.head()

,duration_ms,explicit,track_id,track,popularity,album,album_release_date,album_total_tracks,artist,artist_id
0,63817,True,3iBtk9V7k32rkvE9bSKJWc,gonmakit,45,gonmakit,2024-05-10,1,Brennan Jones,4PaHlM2H2hC3VQV4BfLo4B
1,218500,False,3iRoXGFrF8O0jTul6LAmus,Under Your Spell,78,April Mixtape 3,2023-05-05,16,Snow Strippers,6TsAG8Ve1icEC8ydeHm3C8
2,201034,False,3C4qukRl7kX1wdevwbOudf,Vehicle Dissolve,33,Body Jumper,2021-08-13,13,Provoker,62s5q1tm3UAcRnbGY9CCI9
3,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,1tbsp,6G01WYFYF91rjG5LtwMhY4
4,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,Mietze Conte,3ETvRzzQK7ppB28GcAE8rN


In [29]:
df_tracks['album_release_date'] = df_tracks['album_release_date'].apply(lambda x: f'{x}-01-01' if len(x) == 4 else x)
df_tracks['album_release_date'] = df_tracks['album_release_date'].apply(lambda x: f'{x}-01' if len(x) == 7 else x)
df_tracks['album_release_date'] = pd.to_datetime(df_tracks['album_release_date'])
df_tracks

,duration_ms,explicit,track_id,track,popularity,album,album_release_date,album_total_tracks,artist,artist_id
0,63817,True,3iBtk9V7k32rkvE9bSKJWc,gonmakit,45,gonmakit,2024-05-10,1,Brennan Jones,4PaHlM2H2hC3VQV4BfLo4B
1,218500,False,3iRoXGFrF8O0jTul6LAmus,Under Your Spell,78,April Mixtape 3,2023-05-05,16,Snow Strippers,6TsAG8Ve1icEC8ydeHm3C8
2,201034,False,3C4qukRl7kX1wdevwbOudf,Vehicle Dissolve,33,Body Jumper,2021-08-13,13,Provoker,62s5q1tm3UAcRnbGY9CCI9
3,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,1tbsp,6G01WYFYF91rjG5LtwMhY4
4,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,Mietze Conte,3ETvRzzQK7ppB28GcAE8rN
...,...,...,...,...,...,...,...,...,...,...
3893,341967,False,7k0mvImLkHXeiaAYDQDnyB,"In Europe, Kinda Sad",15,kanashī,2021-03-17,4,1tbsp,6G01WYFYF91rjG5LtwMhY4
3894,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,39,Im_my_worst_enemy,2021-02-01,9,Harto Falión,0wAEYYaHOqYJhHaSvpZUwP
3895,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,39,Im_my_worst_enemy,2021-02-01,9,evilgiane,4bbAeMRdMbfCLVSlpryAVV
3896,112243,True,7tHXd3CsT7VGZAJ5Jkqt61,nothing like uuu,29,nothing like uuu,2024-04-29,1,Nettspend,2jl4qd6UbzeCmImT4nWbtA


In [30]:
df_features = pd.DataFrame(columns= ['track_id', 'track', 'artist', 'artist_id'])

for track, group in df_tracks.groupby('track'):
  if len(group['artist']) > 1:
    for artist in group['artist'][1:]:
      df_features.loc[len(df_features)] = [group.loc[group['artist'] == artist, 'track_id'].values[0], track, artist, group.loc[group['artist'] == artist, 'artist_id'].values[0]]

df_features

,track_id,track,artist,artist_id
0,0tgAhrvNPhSyMZR0wWbv6Z,#1 Freak (feat. Ty Dolla $ign),Ty Dolla $ign,7c0XG5cIJTrrAgEC3ULPiq
1,5fupW8u36XYYlA9IEQI4LJ,#TWITCHING,DJmegan23,56zdYcwtmdYq3WlBYQeHoA
2,1TkWsnheqK0gBtvh07BtNX,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,Fetty Wap,6PXS4YHDkKvl1wkIl4V8DL
3,1TkWsnheqK0gBtvh07BtNX,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,Rich Homie Quan,5lHRUCqkQZCIWeX7xG4sYT
4,5gmkw2031VVIboYqPIjSWB,1 Scale (feat. G Herbo),G Herbo,5QdEbQJ3ylBnc3gsIASAT5
...,...,...,...,...
1322,4S05mkyTtAiWy5l4umch0X,where will i be,Emmylou Harris,5s6TJEuHTr9GR894wc6VfP
1323,59J5nzL1KniFHnU120dQzt,wokeuplikethis*,Lil Uzi Vert,4O15NlyKLIASxsJ0PrXPfz
1324,11qCHQPkZU9L3vvIYQbEGn,Осенний кот,Стас Экстаз,4ciTih43wrAFNsGK9IB2Qt
1325,3oID9FwJyDT5CUfFlxZClc,ただdance in the dark - Mixed,Peterparker69,1tgZVOBOut0nS8CJ5SIBh5


In [31]:
df_features.drop_duplicates(subset= ['track', 'artist'], inplace= True)
df_features.reset_index(drop= True, inplace= True)
df_features

,track_id,track,artist,artist_id
0,0tgAhrvNPhSyMZR0wWbv6Z,#1 Freak (feat. Ty Dolla $ign),Ty Dolla $ign,7c0XG5cIJTrrAgEC3ULPiq
1,5fupW8u36XYYlA9IEQI4LJ,#TWITCHING,DJmegan23,56zdYcwtmdYq3WlBYQeHoA
2,1TkWsnheqK0gBtvh07BtNX,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,Fetty Wap,6PXS4YHDkKvl1wkIl4V8DL
3,1TkWsnheqK0gBtvh07BtNX,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,Rich Homie Quan,5lHRUCqkQZCIWeX7xG4sYT
4,5gmkw2031VVIboYqPIjSWB,1 Scale (feat. G Herbo),G Herbo,5QdEbQJ3ylBnc3gsIASAT5
...,...,...,...,...
1276,4S05mkyTtAiWy5l4umch0X,where will i be,Emmylou Harris,5s6TJEuHTr9GR894wc6VfP
1277,59J5nzL1KniFHnU120dQzt,wokeuplikethis*,Lil Uzi Vert,4O15NlyKLIASxsJ0PrXPfz
1278,11qCHQPkZU9L3vvIYQbEGn,Осенний кот,Стас Экстаз,4ciTih43wrAFNsGK9IB2Qt
1279,3oID9FwJyDT5CUfFlxZClc,ただdance in the dark - Mixed,Peterparker69,1tgZVOBOut0nS8CJ5SIBh5


In [32]:
df_tracks = df_tracks.drop_duplicates(subset= ['track_id'])
df_tracks.reset_index(drop= True, inplace= True)
df_tracks

,duration_ms,explicit,track_id,track,popularity,album,album_release_date,album_total_tracks,artist,artist_id
0,63817,True,3iBtk9V7k32rkvE9bSKJWc,gonmakit,45,gonmakit,2024-05-10,1,Brennan Jones,4PaHlM2H2hC3VQV4BfLo4B
1,218500,False,3iRoXGFrF8O0jTul6LAmus,Under Your Spell,78,April Mixtape 3,2023-05-05,16,Snow Strippers,6TsAG8Ve1icEC8ydeHm3C8
2,201034,False,3C4qukRl7kX1wdevwbOudf,Vehicle Dissolve,33,Body Jumper,2021-08-13,13,Provoker,62s5q1tm3UAcRnbGY9CCI9
3,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,1tbsp,6G01WYFYF91rjG5LtwMhY4
4,210109,False,0ej3RoZrDPhwQ7A6T8UoIS,Sleeves Touch My Elbows,41,Mosquito Love,2023-08-17,8,1tbsp,6G01WYFYF91rjG5LtwMhY4
...,...,...,...,...,...,...,...,...,...,...
2682,231586,False,6zbCsfnPxrzFIaQf6DV3a5,Give It Up,31,Hovvdy,2024-04-26,19,Hovvdy,59RNNqeEfkq3X5pfOQxZ3C
2683,341967,False,7k0mvImLkHXeiaAYDQDnyB,"In Europe, Kinda Sad",15,kanashī,2021-03-17,4,1tbsp,6G01WYFYF91rjG5LtwMhY4
2684,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,39,Im_my_worst_enemy,2021-02-01,9,Harto Falión,0wAEYYaHOqYJhHaSvpZUwP
2685,112243,True,7tHXd3CsT7VGZAJ5Jkqt61,nothing like uuu,29,nothing like uuu,2024-04-29,1,Nettspend,2jl4qd6UbzeCmImT4nWbtA


In [33]:
df_tracks.to_csv('spotify_top_tracks.csv')
df_artists.to_csv('spotify_top_artists.csv')
df_features.to_csv('spotify_top_featured_artists.csv')